# Preprocesamiento (nulos, categóricas y escalado)

## Importar librerías y cargar datos

In [1]:
import pandas as pd
import numpy as np

# Importaremos las clases estándar en scikit-learn para imputación, codificación y escalado.
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

## Cargar el CSV

In [2]:
df = pd.read_csv("Recursos/02-clientes_preprocesamiento.csv")
df.head()

,id,edad,ciudad,ingresos_mensuales,gasto_ultimo_mes,plan,segmento,churn
0,1,56.0,Concepción,1307317.0,237628.0,Premium,Baja,1
1,2,69.0,NaN,721184.0,423632.0,Standard,Media,0
2,3,46.0,La Serena,2247191.0,184415.0,Premium,Baja,1
3,4,32.0,Santiago,1871091.0,456716.0,Premium,Baja,0
4,5,60.0,Santiago,1068234.0,496438.0,Premium,Alta,0


In [3]:
df.tail()

,id,edad,ciudad,ingresos_mensuales,gasto_ultimo_mes,plan,segmento,churn
115,116,52.0,Santiago,1443233.0,88102.0,NaN,Alta,1
116,117,50.0,Santiago,2111741.0,124460.0,Basic,Media,0
117,118,22.0,NaN,929525.0,414778.0,Basic,Media,0
118,119,59.0,Concepción,1468632.0,139930.0,Standard,Baja,0
119,120,56.0,La Serena,576615.0,581089.0,Standard,Alta,0


## Datos Nulos

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  120 non-null    int64  
 1   edad                113 non-null    float64
 2   ciudad              108 non-null    object 
 3   ingresos_mensuales  102 non-null    float64
 4   gasto_ultimo_mes    107 non-null    float64
 5   plan                107 non-null    object 
 6   segmento            120 non-null    object 
 7   churn               120 non-null    int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 7.6+ KB


### Contar los datos Nulos
* isnull() / isna() detectan nulos.
* .sum() suma la cantidad de nulos.

In [5]:
df.isnull().sum()

id                     0
edad                   7
ciudad                12
ingresos_mensuales    18
gasto_ultimo_mes      13
plan                  13
segmento               0
churn                  0
dtype: int64

## Tratar nulos numéricos
* Vamos a trabajar con subset numérico:

In [6]:
columnas_numericas = ["edad", "ingresos_mensuales", "gasto_ultimo_mes"]

df_numerico = df[columnas_numericas]
df_numerico.head()

,edad,ingresos_mensuales,gasto_ultimo_mes
0,56.0,1307317.0,237628.0
1,69.0,721184.0,423632.0
2,46.0,2247191.0,184415.0
3,32.0,1871091.0,456716.0
4,60.0,1068234.0,496438.0


In [7]:
# rellenamos los nulos con el promedio de cada columna.
imputer_media = SimpleImputer(strategy="mean")

# Devuelve una copia de df_numerico donde: Cada NaN es reemplazada por la media.
df_numerico_imputado = imputer_media.fit_transform(df_numerico)

# Mostramos las primeras 5 filas del array.
df_numerico_imputado[:5]

array([[5.600000e+01, 1.307317e+06, 2.376280e+05],
       [6.900000e+01, 7.211840e+05, 4.236320e+05],
       [4.600000e+01, 2.247191e+06, 1.844150e+05],
       [3.200000e+01, 1.871091e+06, 4.567160e+05],
       [6.000000e+01, 1.068234e+06, 4.964380e+05]])

In [8]:
# Como df_numerico_imputado es Arrays, debemos convertirlo a DataFrame
# df_numerico_imputado ya no contiene NULOS y lo cargaremos a la columnas_numericas["edad", "ingresos_mensuales", "gasto_ultimo_mes"]
df_numerico_imputado = pd.DataFrame(
    df_numerico_imputado,
    columns=columnas_numericas
)
df_numerico_imputado.head()

,edad,ingresos_mensuales,gasto_ultimo_mes
0,56.0,1307317.0,237628.0
1,69.0,721184.0,423632.0
2,46.0,2247191.0,184415.0
3,32.0,1871091.0,456716.0
4,60.0,1068234.0,496438.0


## Tratar nulos categóricos

In [9]:
columnas_categoricas = ["ciudad", "plan"]

df_cat = df[columnas_categoricas]
df_cat.head()

,ciudad,plan
0,Concepción,Premium
1,NaN,Standard
2,La Serena,Premium
3,Santiago,Premium
4,Santiago,Premium


In [10]:
# strategy="most_frequent" rellena con la categoría más repetida en cada columna, recomendado para categóricas.
imputer_moda = SimpleImputer(strategy="most_frequent")

df_cat_imputado = imputer_moda.fit_transform(df_cat)

df_cat_imputado[:5]

array([['Concepción', 'Premium'],
       ['Santiago', 'Standard'],
       ['La Serena', 'Premium'],
       ['Santiago', 'Premium'],
       ['Santiago', 'Premium']], dtype=object)

In [11]:
df_cat_imputado = pd.DataFrame(
    df_cat_imputado,
    columns=columnas_categoricas
)

df_cat_imputado.head()

,ciudad,plan
0,Concepción,Premium
1,Santiago,Standard
2,La Serena,Premium
3,Santiago,Premium
4,Santiago,Premium


## Reconstruir un DataFrame limpio

In [12]:
df_limpio = df.copy()
df_limpio[columnas_numericas] = df_numerico_imputado
df_limpio[columnas_categoricas] = df_cat_imputado
df_limpio.head()

,id,edad,ciudad,ingresos_mensuales,gasto_ultimo_mes,plan,segmento,churn
0,1,56.0,Concepción,1307317.0,237628.0,Premium,Baja,1
1,2,69.0,Santiago,721184.0,423632.0,Standard,Media,0
2,3,46.0,La Serena,2247191.0,184415.0,Premium,Baja,1
3,4,32.0,Santiago,1871091.0,456716.0,Premium,Baja,0
4,5,60.0,Santiago,1068234.0,496438.0,Premium,Alta,0


In [13]:
df_limpio.tail()

,id,edad,ciudad,ingresos_mensuales,gasto_ultimo_mes,plan,segmento,churn
115,116,52.0,Santiago,1443233.0,88102.0,Basic,Alta,1
116,117,50.0,Santiago,2111741.0,124460.0,Basic,Media,0
117,118,22.0,Santiago,929525.0,414778.0,Basic,Media,0
118,119,59.0,Concepción,1468632.0,139930.0,Standard,Baja,0
119,120,56.0,La Serena,576615.0,581089.0,Standard,Alta,0


In [14]:
df_limpio.isnull().sum()

id                    0
edad                  0
ciudad                0
ingresos_mensuales    0
gasto_ultimo_mes      0
plan                  0
segmento              0
churn                 0
dtype: int64

## Codificar variables categóricas (One-Hot)
* Usaremos OneHotEncoder sobre ciudad, plan y segmento para convertirlos en columnas numéricas.

In [15]:
# Definimos las columnas
columnas_categoricas_modelo = ["ciudad", "plan", "segmento"]

# Inicializamos el Codificador OneHotEncoder
# sparse_output=False: Indica que la salida de la transformación debe ser un arreglo denso de NumPy (matriz).
# handle_unknown="ignore": evita reventar si aparece una categoría nueva en producción.
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Se llama al método fit_transform aplicando la operación sobre las columnas seleccionadas del DataFrame df_limpio.
# El codificador aprende (o "ajusta") todas las categorías únicas presentes en las tres columnas("ciudad", "plan", "segmento").
# Basándose en las categorías aprendidas, el codificador crea una nueva columna binaria (con valores 0 o 1) por cada categoría única que encontró. 
cat_codificada = encoder.fit_transform(df_limpio[columnas_categoricas_modelo])

cat_codificada[:5]

array([[1., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1., 0., 1., 0., 0.]])

In [16]:
# Ahora conviertes a DataFrame:
nombres_columnas_ohe = encoder.get_feature_names_out(columnas_categoricas_modelo)

df_cat_codificada = pd.DataFrame(
    cat_codificada,
    columns=nombres_columnas_ohe,
    index=df_limpio.index
)

df_cat_codificada.head()

,ciudad_Concepción,ciudad_La Serena,ciudad_Santiago,ciudad_Valparaíso,plan_Basic,plan_Premium,plan_Standard,segmento_Alta,segmento_Baja,segmento_Media
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


## Unir numéricas + categóricas codificadas

In [17]:
columnas_numericas_modelo = ["edad", "ingresos_mensuales", "gasto_ultimo_mes"]

X_features = pd.concat(
    [df_limpio[columnas_numericas_modelo], df_cat_codificada],
    axis=1
)

X_features.head()

,edad,ingresos_mensuales,gasto_ultimo_mes,ciudad_Concepción,ciudad_La Serena,ciudad_Santiago,ciudad_Valparaíso,plan_Basic,plan_Premium,plan_Standard,segmento_Alta,segmento_Baja,segmento_Media
0,56.0,1307317.0,237628.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,69.0,721184.0,423632.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,46.0,2247191.0,184415.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,32.0,1871091.0,456716.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,60.0,1068234.0,496438.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


* Esto ya es una matriz de features numéricos lista para escalar para luego poderla usar en modelos

## Escalado de variables numéricas
* Muchos modelos funcionan mejor si las cateforias o features están normalizadas, por eso aplicas StandardScaler o MinMaxScaler según el caso.

### StandardScaler
* Transforma cada variable numérica para que tenga media 0 y desviación estándar 1.

In [18]:
# Creamos una instancia del objeto StandardScaler.
scaler_std = StandardScaler()

# Selecciona las columnas numéricas del DataFrame.
X_numerico = df_limpio[columnas_numericas_modelo]

# Calcula la media y desviación estándar y luego transforma los datos (transform).
X_numerico_std = scaler_std.fit_transform(X_numerico)

# # Muestra los primeros 5 registros de los datos estandarizados.
X_numerico_std[:5]

array([[ 0.90178479, -0.22349767, -0.53229592],
       [ 1.78316993, -1.29208538,  0.61090439],
       [ 0.22379623,  1.49000046, -0.85934867],
       [-0.72538777,  0.80432701,  0.81424217],
       [ 1.17298022, -0.65937341,  1.05837779]])

In [19]:
X_numerico_std_df = pd.DataFrame(
    X_numerico_std,
    columns=[col + "_std" for col in columnas_numericas_modelo]
)

X_numerico_std_df.describe()

,edad_std,ingresos_mensuales_std,gasto_ultimo_mes_std
count,1.200000e+02,1.200000e+02,1.200000e+02
mean,-1.924387e-16,-3.552714e-16,4.810966e-17
std,1.004193e+00,1.004193e+00,1.004193e+00
min,-1.674572e+00,-1.872078e+00,-1.665398e+00
25%,-7.931866e-01,-6.602430e-01,-8.058545e-01
50%,-4.739920e-02,-4.244770e-16,0.000000e+00
75%,9.017848e-01,7.283637e-01,7.859255e-01
max,1.783170e+00,1.891392e+00,1.692662e+00


* las medias quedan cerca de 0 y la desviación cerca de 1
  - mean ≈ 0
  - std ≈ 1.004
  - min ≈ -1.67
  - max ≈ 1.89

### MinMaxScaler
* reescala cada variable numérica a un rango entre 0 y 1 usando su mínimo y máximo.

In [20]:
scaler_mm = MinMaxScaler()

X_numerico_mm = scaler_mm.fit_transform(X_numerico)

X_numerico_mm_df = pd.DataFrame(
    X_numerico_mm,
    columns=[col + "_mm" for col in columnas_numericas_modelo]
)

X_numerico_mm_df.describe()

,edad_mm,ingresos_mensuales_mm,gasto_ultimo_mes_mm
count,120.000000,120.000000,120.000000
mean,0.484296,0.497434,0.495941
std,0.290419,0.266826,0.299040
min,0.000000,0.000000,0.000000
25%,0.254902,0.321999,0.255964
50%,0.470588,0.497434,0.495941
75%,0.745098,0.690969,0.729982
max,1.000000,1.000000,1.000000


* Las columnas deberían quedar aproximadamente en rango [0, 1].